# Demo of Classless ProBP

Based on Alexander Papen & Bazyli Szymański [2020] "Classless Continuous RM: Exploration of
Methods and Assumptions"


In [242]:
# Basic structures
from math import exp, log, sqrt
from dataclasses import dataclass, field
import pandas as pd

@dataclass
class Item:
    item_no: int
    path_index: int
    tf: int
    mkt: str
    contrib: float
    fcst_mean: float
    fcst_std_dev: float
    

@dataclass
class Leg:
    flt_no: int
    orig: str
    dest: str
    bid_price: float = 1.0
    capacity: int = 10
    items: list[item] = field(default_factory=list)
    
    def str_items(self):
        return pd.DataFrame(self.items)


@dataclass
class Path:
    path_no: int
    orig: str
    dest: str
    leg_index: list[int]
    q_demand: list[float]
    q_fare: float
    y_fare: float
    sum_bp: float = 0.0
    q_std_dev: list[float] = None
    f_star: list[float] = None
    d_star: list[float] = None

In [255]:
# Setup the example from the presentation

def setup():
    num_tf = 4
    frat5 = [1.5, 2.0, 2.5, 3.0]
    k_factor = 0.5
    
    legs = [
        Leg(1, "AAA", "BBB"),
        Leg(2, "BBB", "CCC")
    ]
    
    paths = [
        Path(1, "AAA", "BBB", [0], [5, 5, 5, 5], 100.0, 800.0),
        Path(2, "BBB", "CCC", [1], [7, 7, 7, 7], 75.0, 600.0),
        Path(3, "AAA", "CCC", [0, 1], [5, 5, 5, 5], 125.0, 1000.0)
    ]

    for p in paths:
        p.f_star = [0.0] * num_tf
        p.d_star = [0.0] * num_tf
        p.q_std_dev = [round(d * k_factor, 2) for d in p.q_demand]

    return legs, paths
    

In [250]:
pd.DataFrame(paths)

,path_no,orig,dest,leg_index,q_demand,q_fare,y_fare,sum_bp,q_std_dev,f_star,d_star
0,1,AAA,BBB,[0],"[5, 5, 5, 5]",100.0,800.0,0.0,None,None,None
1,2,BBB,CCC,[1],"[7, 7, 7, 7]",75.0,600.0,0.0,None,None,None
2,3,AAA,CCC,"[0, 1]","[5, 5, 5, 5]",125.0,1000.0,0.0,None,None,None


In [208]:
# Some functions for Classless ProBP



In [258]:
# STEP 1
def calculate_fare():
    for p in paths:
        for t in range(num_tf):
            p.sum_bp = 0.0
            for l in p.leg_index:
                p.sum_bp += legs[l].bid_price
            tmp = p.q_fare * (frat5[t] - 1.0) / log(2.0)
            p.f_star[t] = round(min(p.y_fare, max(p.q_fare, tmp) + p.sum_bp), 2)
            p_wtp = min(1.0, exp((log(0.5) * (p.f_star[t] - p.q_fare)) / ((frat5[t] - 1.0) * p.q_fare)))
            p.d_star[t] = round(p.q_demand[t] * p_wtp, 2)

In [260]:
# Proration !!!
def proration(debug=False):
    for leg in legs:
        leg.items = []
    
    for p in paths:
        mkt = f"{p.orig}-{p.dest}"
        for l in p.leg_index:
            leg = legs[l]
            for t in range(num_tf):
                contrib = p.f_star[t] * leg.bid_price / p.sum_bp
                mean = p.d_star[t]
                var = p.q_demand[t] ** 2
                var *= mean / p.q_demand[t]
                std_dev = sqrt(var)
                item = Item(0, p.path_no, t, mkt, contrib, mean, std_dev)
                leg.items.append(item)
    
    # Sort the items by contribution
    for leg in legs:
        leg.items.sort(reverse=True, key=lambda x: x.contrib)
        if debug:
            print(f"Leg: {leg.orig}-{leg.dest}")
            print("    ", legs[0].str_items())


In [261]:
# EMSRc calculations
from statistics import NormalDist
nd = NormalDist(mu=0.0, sigma=1.)

def emsrc(debug=False):
    for leg in legs:
        if debug:
            print(f"EMSRc, Leg = {leg.orig}-{leg.dest}")
        agg_mean, agg_variance, agg_fare = 0, 0, 0
        prot = 0.0
        bid_price = 0.0
        prior_avg_fare = 0.0
        prior_agg_mean = 0.0
        prior_agg_stdev = 0.0
        for item in leg.items:
            if prior_avg_fare > 0:
                fare_ratio = item.contrib / prior_avg_fare
                fare_ratio = min(0.99999, max(0.00001, fare_ratio))
                z_score = NormalDist(mu=0.0, sigma=1.0).inv_cdf(fare_ratio)
                prot = prior_agg_mean + z_score * prior_agg_stdev
                if prior_protection_level < leg.capacity:
                    raw_bp = (1 - nd.cdf((leg.capacity - prior_agg_mean) / prior_agg_stdev)) * prior_avg_fare
                    bid_price = max(raw_bp, 0)
                if debug:
                    print(f"    μ={item.fcst_mean}, σ={round(item.fcst_std_dev,2)}, bp={round(bid_price,2)}")
                    
            agg_mean += item.fcst_mean
            agg_variance += item.fcst_std_dev ** 2
            agg_fare += item.contrib * item.fcst_mean
    
            prior_agg_mean = agg_mean
            prior_agg_stdev = sqrt(agg_variance)
            prior_avg_fare =  agg_fare / agg_mean if agg_mean > 0 else 0.0;
            prior_protection_level = prot
            prior_contributed_fare = item.contrib
            if debug:
                print(f"    agg_mean={agg_mean}, agg_variance={round(agg_variance,2)}, agg_fare={agg_fare}")
        leg.bid_price = bid_price
        

In [256]:
legs, paths = setup()

In [263]:
calculate_fare()
pd.DataFrame(paths)

,path_no,orig,dest,leg_index,q_demand,q_fare,y_fare,sum_bp,q_std_dev,f_star,d_star
0,1,AAA,BBB,[0],"[5, 5, 5, 5]",100.0,800.0,1.0,"[2.5, 2.5, 2.5, 2.5]","[101.0, 145.27, 217.4, 289.54]","[4.93, 3.65, 2.91, 2.59]"
1,2,BBB,CCC,[1],"[7, 7, 7, 7]",75.0,600.0,1.0,"[3.5, 3.5, 3.5, 3.5]","[76.0, 109.2, 163.3, 217.4]","[6.87, 5.1, 4.06, 3.63]"
2,3,AAA,CCC,"[0, 1]","[5, 5, 5, 5]",125.0,1000.0,2.0,"[2.5, 2.5, 2.5, 2.5]","[127.0, 182.34, 272.51, 362.67]","[4.89, 3.64, 2.9, 2.59]"


In [272]:
for n in range(5):
    calculate_fare()
    proration()
    emsrc()
    print(f"***** Iteration: {n}")
    print(pd.DataFrame(legs)[["flt_no", "orig", "dest", "bid_price"]])

info = []
for p in paths:
    for tf, f in enumerate(p.f_star):
        info.append({"orig": p.orig,
                     "dest": p.dest,
                     "tf": tf,
                     "f_star": f})
print("***** Paths and fares")
print(pd.DataFrame(info))

***** Iteration: 0
   flt_no orig dest  bid_price
0       1  AAA  BBB  99.171494
1       2  BBB  CCC  59.823400
***** Iteration: 1
   flt_no orig dest  bid_price
0       1  AAA  BBB  99.171494
1       2  BBB  CCC  59.823400
***** Iteration: 2
   flt_no orig dest  bid_price
0       1  AAA  BBB  99.171494
1       2  BBB  CCC  59.823400
***** Iteration: 3
   flt_no orig dest  bid_price
0       1  AAA  BBB  99.171494
1       2  BBB  CCC  59.823400
***** Iteration: 4
   flt_no orig dest  bid_price
0       1  AAA  BBB  99.171494
1       2  BBB  CCC  59.823400
***** Paths and fares
   orig dest  tf  f_star
0   AAA  BBB   0  199.17
1   AAA  BBB   1  243.44
2   AAA  BBB   2  315.58
3   AAA  BBB   3  387.71
4   BBB  CCC   0  134.82
5   BBB  CCC   1  168.03
6   BBB  CCC   2  222.13
7   BBB  CCC   3  276.23
8   AAA  CCC   0  283.99
9   AAA  CCC   1  339.33
10  AAA  CCC   2  429.50
11  AAA  CCC   3  519.67


In [267]:
pd.DataFrame(legs)

,flt_no,orig,dest,bid_price,capacity,items
0,1,AAA,BBB,99.127528,10,"[{'item_no': 0, 'path_index': 1, 'tf': 3, 'mkt..."
1,2,BBB,CCC,59.714965,10,"[{'item_no': 0, 'path_index': 2, 'tf': 3, 'mkt..."
